In [2]:
import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
# from langchain.document_loaders import PyPDFLoader
from langchain.llms import VertexAI

In [4]:
!pip install google-cloud-aiplatform>=1.38.0

zsh:1: 1.38.0 not found


# Using MapReduce 

In [5]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [ ]:
map_reduce_chain = load_summarize_chain(
    vertex_llm_text,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": pages})

# Sentece Transformers to Optimize Model 

In [7]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Synonym dictionary for tag normalization 
synonym_dict = {
    "key": ["value"]
}

In [ ]:
# Normalized Tags
print(precompute_synonym_embeddings(synonym_dict, model))